<a href="https://colab.research.google.com/github/anaswaraku/Kidney-disease-detection/blob/main/DenseNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Activation, Conv2D, Concatenate, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator



In [10]:
# Define DenseNet Block
def dense_block(x, growth_rate, num_layers):
    for _ in range(num_layers):
        # Bottleneck layer (1x1 convolution)
        inter_channel = 4 * growth_rate
        x1 = BatchNormalization()(x)
        x1 = Activation('relu')(x1)
        x1 = Conv2D(inter_channel, kernel_size=1, padding='same', use_bias=False)(x1)

        # Convolution layer (3x3 convolution)
        x1 = BatchNormalization()(x1)
        x1 = Activation('relu')(x1)
        x1 = Conv2D(growth_rate, kernel_size=3, padding='same', use_bias=False)(x1)

        # Concatenate output
        x = Concatenate(axis=-1)([x, x1])
    return x


In [11]:

# Define Transition Block
def transition_block(x, compression_factor):
    num_channels = int(x.shape[-1])
    num_channels = int(num_channels * compression_factor)

    x = BatchNormalization()(x)
    x = Conv2D(num_channels, kernel_size=1, padding='same', use_bias=False)(x)
    x = Activation('relu')(x)
    x = tf.keras.layers.AveragePooling2D(pool_size=2, strides=2, padding='same')(x)
    return x


In [12]:

# DenseNet model
def DenseNet(input_shape, num_classes, growth_rate, num_layers_per_block, compression_factor):
    inputs = Input(shape=input_shape)
    x = Conv2D(64, kernel_size=7, strides=2, padding='same', use_bias=False)(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = tf.keras.layers.MaxPooling2D(pool_size=3, strides=2, padding='same')(x)

    for i, num_layers in enumerate(num_layers_per_block):
        # Dense Block
        x = dense_block(x, growth_rate, num_layers)

        # Transition Block
        if i != len(num_layers_per_block) - 1:
            x = transition_block(x, compression_factor)

    # Global average pooling and classification layer
    x = GlobalAveragePooling2D()(x)
    outputs = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=inputs, outputs=outputs)
    return model


In [13]:

# Parameters
input_shape = (224, 224, 3)  # adjust according to your preprocessed image size
num_classes = 4  # Number of classes: cyst, tumor, stone, normal
growth_rate = 32  # Growth rate parameter (K)
num_layers_per_block = [6, 12, 24, 16]  # Number of layers in each dense block
compression_factor = 0.5  # Compression factor in transition blocks

# Create DenseNet model
model = DenseNet(input_shape, num_classes, growth_rate, num_layers_per_block, compression_factor)

model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])



In [14]:

# Data Augmentation and Image Preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)


In [15]:

# Load the dataset
train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/Preprocessed/train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)


Found 9791 images belonging to 4 classes.


In [ ]:

# Train the model
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples/train_generator.batch_size,
    epochs=20  # adjust as needed
)

# Save the model
model.save('denseNet_model.h5')


Epoch 1/20
 12/305 [>.............................] - ETA: 2:01:45 - loss: 3.0195 - accuracy: 0.2344